In [1]:
import torch
import numpy as np
from facenet_pytorch import InceptionResnetV1

# Load the model architecture 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = InceptionResnetV1(pretrained='vggface2').to(device)  

# Load the pre-trained model's state_dict (weights)
model.load_state_dict(torch.load(r"D:\2nd_april\less_dataset_model.pth", map_location=device))
model.eval()  # Set the model to evaluation mode

# Load the known face embeddings and names
known_face_encodings = np.load(r"D:\2nd_april\My_embeddings.npy")
known_face_names = np.load(r"D:\2nd_april\My_labels.npy")

# Print to confirm successful loading
print(f"Model loaded successfully{model}")
# print(f"Loaded {len(known_face_encodings)} embeddings.")
# print(f"Loaded {len(known_face_names)} names.")


Model loaded successfullyInceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): R

In [2]:
import torch
import os
import pandas as pd
import numpy as np
import cv2
from mtcnn import MTCNN  
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image

import sys
from facenet_pytorch import InceptionResnetV1
from facenet_pytorch import InceptionResnetV1, MTCNN
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
import os
import pandas as pd
from datetime import datetime, time  # Import missing datetime and time

attendance_file = "attendance.xlsx"  # Define the file path



In [4]:


# Create an empty DataFrame with correct column names
df = pd.DataFrame(columns=["Name", "Date", "Time", "Subject"])
df.to_excel("attendance.xlsx", index=False)


Model loaded successfully.


In [18]:
import cv2
import numpy as np
import torch
import os
import time
import pandas as pd
from datetime import datetime, time as dt_time
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

# Paths to model and embeddings
model_path = r"D:\2nd_april\less_dataset_model.pth"
embedding_path = r"D:\2nd_april\My_embeddings.npy"
labels_path = r"D:\2nd_april\My_labels.npy"
attendance_file = "attendance.xlsx"

# Initialize MTCNN and FaceNet model
mtcnn = MTCNN(keep_all=True)
model = InceptionResnetV1(pretrained='vggface2').eval()
model.load_state_dict(torch.load(model_path))
model.eval()

print("✅ Model loaded successfully.")

# Load embeddings and labels
embeddings = np.load(embedding_path)
labels = np.load(labels_path)
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
decoded_labels = label_encoder.inverse_transform(encoded_labels)

# Create attendance file if missing
def initialize_attendance_file():
    if not os.path.exists(attendance_file):
        df = pd.DataFrame(columns=["Name", "Date", "Time", "Subject"])
        df.to_excel(attendance_file, index=False)
        print("✅ Created attendance.xlsx file.")

# Save attendance with error handling
def save_attendance(df):
    try:
        df.to_excel(attendance_file, index=False)
    except Exception as e:
        print(f"⚠️ Error saving attendance: {e}")

# Load attendance file
def load_attendance():
    if os.path.exists(attendance_file):
        return pd.read_excel(attendance_file)
    return pd.DataFrame(columns=["Name", "Date", "Time", "Subject"])

# Function to check if a student is already marked present
def is_already_marked(name, subject):
    df = load_attendance()
    today = datetime.today().strftime("%Y-%m-%d")
    return ((df["Name"] == name) & (df["Date"] == today) & (df["Subject"] == subject)).any()

# Get the current timetable slot
def get_current_slot():
    timetable = {
        "Monday": [(dt_time(9,30), dt_time(11,10), "Java"),
                   (dt_time(11,10), dt_time(12,0), "CN"),
                   (dt_time(12,0), dt_time(12,50), "VA"),
                   (dt_time(13,35), dt_time(14,20), "Big Data Analytics"),
                   (dt_time(14,20), dt_time(15,5), "Neural Network"),
                   (dt_time(15,5), dt_time(15,50), "MPMC"),
                   (dt_time(15,50), dt_time(16,35), "Coding Practice/Project")],

        "Tuesday": [(dt_time(9,30), dt_time(11,10), "Java"),
                    (dt_time(11,10), dt_time(12,0), "Machine Learning Manjot"),
                    (dt_time(12,0), dt_time(12,50), "MPMC"),
                    (dt_time(13,35), dt_time(15,5), "Computer Network lab"),
                    (dt_time(15,5), dt_time(15,50), "Machine Learning")],
        
        "Wednesday": [(dt_time(9,30), dt_time(11,10), "ML & Big Data Analytics"),
                      (dt_time(11,10), dt_time(12,50), "Java"),  
                      (dt_time(13,35), dt_time(15,5), "Neural Networks Lab")],

        "Thursday": [(dt_time(9,30), dt_time(10,20), "Soft Skills"),
                     (dt_time(10,20), dt_time(11,10), "CN"),
                     (dt_time(11,10), dt_time(12,0), "LR"),
                     (dt_time(12,0), dt_time(12,50), "Big Data Analytics"),
                     (dt_time(13,35), dt_time(14,20), "Machine Learning"),
                     (dt_time(14,20), dt_time(15,5), "Neural Network")],

        "Friday": [(dt_time(9,30), dt_time(10,20), "CN"),
                   (dt_time(10,20), dt_time(11,10), "Big Data Analytics"),
                   (dt_time(11,10), dt_time(12,0), "QA"),
                   (dt_time(12,0), dt_time(12,50), "MPMC"),
                   (dt_time(13,35), dt_time(14,20), "Neural Network"),
                   (dt_time(14,20), dt_time(15,5), "Machine Learning"),
                   (dt_time(15,5), dt_time(16,35), "ML Group lab")]
    }

    current_time = datetime.now().time()
    today = datetime.today().strftime("%A")

    for start, end, subject in timetable.get(today, []):
        if start <= current_time <= end:
            return subject
    return None

# Get embedding from image
def get_embedding(image):
    faces = mtcnn(image)
    if faces is not None and len(faces) > 0:
        return model(faces).detach().cpu().numpy()
    return None

# Compute similarity
def compute_similarity(embedding1, embedding2):
    return cosine_similarity(embedding1.reshape(1, -1), embedding2.reshape(1, -1))[0][0]

# Compare embeddings
def compare_embeddings(test_embedding):
    if test_embedding is None:
        return "No face detected", 0

    similarities = [compute_similarity(test_embedding, emb) for emb in embeddings]
    max_similarity = np.max(similarities)

    if max_similarity >= 0.6:  # Threshold
        return labels[np.argmax(similarities)], max_similarity
    return "No match found", max_similarity

# Start attendance system
def mark_attendance():
    initialize_attendance_file()
    cap = cv2.VideoCapture(0)
    start_time = time.time()
    last_recorded_time = {}

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Auto-exit after 2 minutes
        if time.time() - start_time >= 120:
            print("⏳ Time limit reached. Exiting...")
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb_frame_pil = Image.fromarray(rgb_frame)

        boxes, _ = mtcnn.detect(rgb_frame_pil)

        if boxes is not None:
            for box in boxes:
                x1, y1, x2, y2 = map(int, box)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                face = rgb_frame_pil.crop((x1, y1, x2, y2))
                face_embedding = get_embedding(face) if face else None

                label, similarity = compare_embeddings(face_embedding)

                if label != "No match found":
                    cv2.putText(frame, f"{label} ({similarity:.2f})", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    subject = get_current_slot()
                    current_time = datetime.now()

                    if subject and (label not in last_recorded_time or (current_time - last_recorded_time[label]).total_seconds() >= 30):
                        last_recorded_time[label] = current_time

                        if not is_already_marked(label, subject):
                            df = load_attendance()
                            new_entry = pd.DataFrame([[label, current_time.strftime("%Y-%m-%d"), current_time.strftime("%H:%M:%S"), subject]],
                                                     columns=["Name", "Date", "Time", "Subject"])
                            df = pd.concat([df, new_entry], ignore_index=True)
                            save_attendance(df)
                            print(f"✅ {label} marked present for {subject} at {current_time.strftime('%H:%M:%S')}")

        cv2.imshow("Attendance System", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    mark_attendance()


✅ Model loaded successfully.
✅ No face detected marked present for Neural Network at 14:30:11
